In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from skimage import data,filters,segmentation,morphology,exposure
from skimage import io as skio

In [ ]:
# Parameters figures
plt.rcParams['figure.figsize'] = (15.0, 8.0)
#plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
#plt.rcParams['figure.dpi'] = 600

# Read image from confocal

In [ ]:
# Read tiff image

from skimage import io as skio

#Original image
img = skio.imread("figs/NHAwt_NaCl_c1_488nm_60x_4x10ux_RPM180-1.tif")
#corrected exposure mixture (High contrast)
img_corrected_exposure = exposure.adjust_log(img, gain=30)
#img_corrected_exposure = exposure.adjust_gamma(img, gain=100)

f, axarr = plt.subplots(nrows=1, ncols=2, figsize=(12.0, 12.0))
axarr[0].set_title('Original')
axarr[0].imshow(img)
axarr[1].set_title('Logarithmic correction')
axarr[1].imshow(img_corrected_exposure)
plt.show()

In [ ]:
hist = exposure.histogram(img, nbins=10)
hist_corrected_exposure = exposure.histogram(img_corrected_exposure, nbins=10)
f, axarr = plt.subplots(nrows=1, ncols=2)
axarr[0].plot(hist[1], hist[0], 'o')
axarr[0].set_xscale('log')
axarr[0].set_yscale('log')
axarr[0].axis((100,60000,1,None))
axarr[0].set_title('Histogram original')

axarr[1].plot(hist_corrected_exposure[1], hist_corrected_exposure[0], 'o')
axarr[1].set_xscale('log')
axarr[1].set_yscale('log')
axarr[1].axis((100,60000,1,None))
axarr[1].set_title('Histogram adjusted exposure')
plt.show()

# Details Image read

In [ ]:
import exifread
# Open image file for reading (binary mode)
f = open("figs/NHAwt_NaCl_c1_488nm_60x_4x10ux_RPM180-1.tif", 'rb')

# Return Exif tags
tags = exifread.process_file(f)

# Print the tag/ value pairs
for tag in tags.keys():
    if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote', 'Image ImageDescription'):
        print("Key: {0}, value {1}".format(tag, tags[tag]))
        

print("\nImage ImageDescription")
print(tags["Image ImageDescription"].values)
#tags_d = tags["Image ImageDescription"]
#print(type(tags_d))
#print(dir(tags_d))

In [ ]:
# General
print("shape of image: {0}".format(img.shape))
print("realsize 26.4 x 26.5 micrometers; 16 bit, 800K")
print("dtype of image: {0}".format(img.dtype))

# Original image
print("\nORIGINAL")
print("--------")
print("Min value pixel = {0}".format(img.min()))
print("Max value pixel = {0}".format(img.max()))

#Corrected exposure
print("\nCORRECTED EXPOSURE")
print("--------")
print("Min value pixel = {0}".format(img_corrected_exposure.min()))
print("Max value pixel = {0}".format(img_corrected_exposure.max()))

In [ ]:
manual_background_spots = np.array([[10,10],[600,600]])
manual_membrane_spots  = np.array([[230,500],[215,510]])
manual_interior_spots  = np.array([[270,480],[200,550]])

#Original
print("ORIGINAL")
print("--------")
print("I_background: {}".format(img[tuple(manual_background_spots.T)]))
print("I_membrane  : {}".format(img[tuple(manual_membrane_spots.T)]))
print("I_interior  : {}".format(img[tuple(manual_interior_spots.T)]))


#print(img_corrected_exposure[100][100])
#print(img[100][100])

#Plot results
f, axarr = plt.subplots(nrows=1, ncols=1, figsize=(15,8))
plt.title('light spots in image')
axarr.imshow(img_corrected_exposure)
axarr.plot( manual_background_spots[:, 1], manual_background_spots[:, 0], 'o', color='red')
axarr.plot( manual_membrane_spots[:, 1], manual_membrane_spots[:, 0], 'v', color='yellow')
axarr.plot( manual_interior_spots[:, 1], manual_interior_spots[:, 0], 's', color='pink')
plt.show()

# What now?

We will remove the background using the following approach:

    1. detect edges
    2. seed the obtained image to separate background from foreground
    3. perform a watershed transformation


## Detecting the edges

In [ ]:
sobel = filters.sobel(img)
#sobel = filters.sobel(img_corrected_exposure)
f, axarr = plt.subplots(nrows=1, ncols=2)
#axarr[0].imshow(img)
axarr[0].imshow(img_corrected_exposure)
axarr[0].plot( manual_background_spots[:, 1], manual_background_spots[:, 0], 'o', color='red')
axarr[0].plot( manual_membrane_spots[:, 1], manual_membrane_spots[:, 0], 'v', color='yellow')
axarr[0].plot( manual_interior_spots[:, 1], manual_interior_spots[:, 0], 's', color='pink')
axarr[1].imshow(sobel)
plt.show()

## blur the image a bit to make the edges thicker

In [ ]:
blurred = filters.gaussian(sobel, sigma=2.0)
f, axarr = plt.subplots(nrows=1, ncols=3)
#axarr[0].imshow(img)
axarr[0].imshow(img_corrected_exposure)
axarr[1].imshow(sobel)
axarr[2].imshow(blurred)
plt.show()



# Obtaining seeds for the watershed transform

To perform a watershed transform on this image, we need to assign classes to the fore and background. There's two ways that I know of to do this:

    - use dark and light zone properties of the image
    - use local peaks from the distance transformed image

Both methods are valid. In this case, let's use the simplest one of the two: dark and light zones.

In [ ]:
# Ligt spots
light_spots = np.array((img > 555).nonzero()).T # For original image
#light_spots = np.array((img_corrected_exposure > 65500).nonzero()).T # For img_corrected_exposure

print("Shape array = {0}".format(light_spots.shape))

#Plot results
f, axarr = plt.subplots(nrows=1, ncols=2)
plt.title('light spots in image')
axarr[0].imshow(img)
#axarr[0].imshow(img_corrected_exposure)
axarr[1].plot(light_spots[:, 1], light_spots[:, 0], 'o')
axarr[1].imshow(img)
#axarr[1].imshow(img_corrected_exposure)
plt.show()

In [ ]:
# Dark spots
dark_spots = np.array((img < 245).nonzero()).T
#dark_spots = np.array((img_corrected_exposure < 39016).nonzero()).T

print("Shape array = {0}".format(dark_spots.shape))

#Plot results
f, axarr = plt.subplots(nrows=1, ncols=2)
plt.title('dark spots in image')
axarr[0].imshow(img)
#axarr[0].imshow(img_corrected_exposure)
axarr[1].plot(dark_spots[:, 1], dark_spots[:, 0], 'o')
axarr[1].imshow(img)
#axarr[1].imshow(img_corrected_exposure)
plt.show()

# Applying the watershed with a mask by regions

In [ ]:
seed_mask = np.zeros(img.shape, dtype=np.int)

seed_mask[tuple(dark_spots.T)] = 1 # background
seed_mask[tuple(light_spots.T)] = 2 # foreground


ws = morphology.watershed(blurred, seed_mask)
#ws = segmentation.random_walker(blurred, seed_mask)

#Plot results
f, axarr = plt.subplots(nrows=1, ncols=3)
axarr[0].imshow(img)
axarr[1].imshow(ws)
axarr[2].imshow(img_corrected_exposure)

plt.show()

In [ ]:
background_spots = np.array((img < 245).nonzero()).T
membrane_spots = np.array((img > 780).nonzero()).T # For original image

#Manual:
interior_spots  = np.array([[270,480],[200,550]])



seed_mask = np.zeros(img.shape, dtype=np.int)

seed_mask[tuple(background_spots.T)] = 1 # background
seed_mask[tuple(membrane_spots.T)] = 2 # membrane
seed_mask[tuple(interior_spots.T)] = 3 # interior


ws = morphology.watershed(blurred, seed_mask)
#ws = segmentation.random_walker(blurred, seed_mask)

#Plot results
f, axarr = plt.subplots(nrows=1, ncols=2)
axarr[0].imshow(img)
axarr[1].imshow(ws)
plt.show()